In [ ]:
!wget -N https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv

In [ ]:
!pip install -q git+https://github.com/tensorflow/docs

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
dataset = pd.read_csv('insurance.csv')
len(dataset)

In [ ]:
dataset.head()

In [ ]:
df = dataset
df["sex"] = pd.factorize(df["sex"])[0]
df["region"] = pd.factorize(df["region"])[0]
df["smoker"] = pd.factorize(df["smoker"])[0]
dataset = df
dataset.head()

In [ ]:
test_dataset = dataset.sample(frac=0.2)
len(test_dataset)

In [ ]:
train_dataset = dataset[~dataset.isin(test_dataset)].dropna()
len(train_dataset)

In [ ]:
train_dataset.head()

In [ ]:
train_labels = train_dataset.pop("expenses")
train_labels.head()

In [ ]:
train_dataset.head()

In [ ]:
test_labels = test_dataset.pop("expenses")
test_labels.head()

In [ ]:
test_dataset.head()

In [ ]:
# Instead of using the experimental module, use the preprocessing layer directly from keras.layers
from tensorflow.keras.layers import Normalization

normalizer = Normalization() # Instantiate the Normalization layer
normalizer.adapt(np.array(train_dataset))


In [ ]:
# Instead of using the experimental module, use the preprocessing layer directly from keras.layers
from tensorflow.keras.layers import Normalization, Input
from tensorflow.keras import Model

normalizer = Normalization() # Instantiate the Normalization layer
normalizer.adapt(np.array(train_dataset))

# Define your model
input_layer = Input(shape=(train_dataset.shape[1],))  # Adjust input shape according to your data
normalized_data = normalizer(input_layer)
# ... Add your hidden layers here ... e.g., Dense layers
# The line below was changed. Replace ... with normalized_data
output_layer = layers.Dense(1)(normalized_data)  # Output layer for regression

model = Model(inputs=input_layer, outputs=output_layer)  # Create the Keras model

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mae',
    metrics=['mae', 'mse']
)
model.build(input_shape=(None, train_dataset.shape[1])) # Build the model explicitly
model.summary()



In [ ]:
history = model.fit(
    train_dataset,
    train_labels,
    epochs=100,
    validation_split=0.5,
    verbose=0, # disable logging
)

print(history)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
